In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!unzip '/content/Test_data.zip'

Archive:  /content/Test_data.zip
   creating: Test_data/alluarjun/
  inflating: Test_data/alluarjun/85.jpg  
  inflating: Test_data/alluarjun/86.jpg  
  inflating: Test_data/alluarjun/87.jpg  
  inflating: Test_data/alluarjun/88.jpg  
  inflating: Test_data/alluarjun/89.jpg  
  inflating: Test_data/alluarjun/90.jpg  
  inflating: Test_data/alluarjun/91.jpg  
  inflating: Test_data/alluarjun/92.jpg  
  inflating: Test_data/alluarjun/93.jpg  
  inflating: Test_data/alluarjun/94.jpg  
  inflating: Test_data/alluarjun/95.jpg  
  inflating: Test_data/alluarjun/96.jpg  
  inflating: Test_data/alluarjun/97.jpg  
  inflating: Test_data/alluarjun/98.jpg  
  inflating: Test_data/alluarjun/99.jpg  
   creating: Test_data/manohar/
  inflating: Test_data/manohar/469.jpg  
  inflating: Test_data/manohar/470.jpg  
  inflating: Test_data/manohar/471.jpg  
  inflating: Test_data/manohar/472.jpg  
  inflating: Test_data/manohar/473.jpg  
  inflating: Test_data/manohar/474.jpg  
  inflating: Test_data/ma

In [2]:
!unzip '/content/train_data.zip'

Archive:  /content/train_data.zip
   creating: train_data/alluarjun/
  inflating: train_data/alluarjun/1.jpg  
  inflating: train_data/alluarjun/10.jpg  
  inflating: train_data/alluarjun/100.jpg  
  inflating: train_data/alluarjun/101.jpg  
  inflating: train_data/alluarjun/11.jpg  
  inflating: train_data/alluarjun/12.jpg  
  inflating: train_data/alluarjun/13.jpg  
  inflating: train_data/alluarjun/14.jpg  
  inflating: train_data/alluarjun/15.jpg  
  inflating: train_data/alluarjun/16.jpg  
  inflating: train_data/alluarjun/17.jpg  
  inflating: train_data/alluarjun/18.jpg  
  inflating: train_data/alluarjun/19.jpg  
  inflating: train_data/alluarjun/2.jpg  
  inflating: train_data/alluarjun/20.jpg  
  inflating: train_data/alluarjun/21.jpg  
  inflating: train_data/alluarjun/22.jpg  
  inflating: train_data/alluarjun/23.jpg  
  inflating: train_data/alluarjun/24.jpg  
  inflating: train_data/alluarjun/25.jpg  
  inflating: train_data/alluarjun/26.jpg  
  inflating: train_data/allu

In [2]:
# Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_path = "C:/Users/5593/Downloads/train_data"
test_path = "C:/Users/5593/Downloads/Test_data"

In [4]:
train_gen = ImageDataGenerator(rescale=1./255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True)

test_gen = ImageDataGenerator(rescale=1./255) 

In [5]:
train = train_gen.flow_from_directory(train_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

test = test_gen.flow_from_directory(test_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

Found 902 images belonging to 3 classes.
Found 72 images belonging to 3 classes.


In [6]:
train.class_indices

{'alluarjun': 0, 'manohar': 1, 'mounish': 2}

In [7]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
from keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Dropout

In [8]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [9]:
# Adding the preprocessing layer to the front of vgg

vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

In [10]:
for layer in vgg.layers:
  print(layer)

In [11]:
# Train model with existing weights

for layer in vgg.layers:
  layer.trainable=False

In [12]:
x = Flatten()(vgg.output)

In [13]:


x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)

In [14]:
# output layer
prediction = Dense(3,activation='softmax')(x)

In [15]:
# Create Vgg16 model

model = Model(inputs=vgg.input,outputs=prediction)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
len(train),len(test)

(41, 4)

In [ ]:
model.fit_generator(train,validation_data=test,epochs=6,steps_per_epoch=len(train),
                    validation_steps=len(test))

C:\Users\5593\AppData\Local\Temp\ipykernel_21936\1421131578.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train,validation_data=test,epochs=6,steps_per_epoch=len(train),


In [ ]:
model.save('generated_model.h5')

In [ ]:
img1 = image.load_img('/content/mou_test.jpg',target_size=(224,224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['Alluarjun','Manohar','Mounish']
print(output[pred])

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
from keras.models import load_model
import numpy as np
from PIL import Image, ImageOps

In [4]:
model=load_model("C:/Users/5593/Downloads/generated_model.h5",compile=False)

In [5]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [6]:
facedetect = cv2.CascadeClassifier("D:/AI projects/Face Recognition System/haarcascade_frontalface_default.xml")

cap=cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
font=cv2.FONT_HERSHEY_COMPLEX

In [ ]:
class_names = open("D:/AI projects/Face Recognition System/labels.txt", "r").readlines()

while True:
	sucess, imgOrignal=cap.read()
	faces = facedetect.detectMultiScale(imgOrignal,1.3,5)
	for x,y,w,h in faces:
		image = cv2.resize(imgOrignal, (224, 224), interpolation=cv2.INTER_AREA)
		image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)
		image = (image / 127.5) - 1
		
		prediction = model.predict(image)
		index = np.argmax(prediction)
		class_name = class_names[index]
		confidence_score = prediction[0][index]
		print("Class:", class_name[2:], end="")
		print("Confidence Score:", confidence_score)
		cv2.rectangle(imgOrignal, (x, y), (x + w, y + h), (0, 255, 0), 2)
		cv2.rectangle(imgOrignal, (x, y - 50), (x + w, y), (0, 255, 0), -2)
		cv2.putText(imgOrignal, "Hii "+str(class_name[2:]), (x, y - 5), font, 0.75, (255, 0, 0), 1,cv2.LINE_AA)
		cv2.putText(imgOrignal,str(round(confidence_score*100, 2))+"%" ,(x, y-25), font, 0.75, (255,0,255),1, cv2.LINE_AA)
		# output_video.write(imgOrignal)
	cv2.imshow("Result",imgOrignal)
	k=cv2.waitKey(1)
	if k==ord('q'):
		break


cap.release()
# output_video.release()
cv2.destroyAllWindows()

